**hERG - QSAR**

-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

**Step 1 : Import the libraries** 

In [1]:
import pandas as pd                     # 'panel data' makes easy to manipulate data
import numpy as np                      # to allow to generate random  numbers 

import matplotlib.pyplot as plt                # to draw graphs
import seaborn as sns                          # ...
import plotly.express as px                    # ...

from sklearn.ensemble import RandomForestClassifier                   # to build a random forest classifier 
from sklearn.tree import DecisionTreeClassifier                       # to build a classification tree

import statistics                                                     # to realise statistic calculations
import itertools                                                      # to flatten lists 
from operator import index                                            # to find the index of items in a list

from sklearn.metrics import confusion_matrix                          # to create a confusion matrix
from sklearn.metrics import plot_confusion_matrix                     # to plot confusion matrix

from sklearn.metrics import balanced_accuracy_score                   # to load metrics
from sklearn.metrics import matthews_corrcoef                         # ... 

from sklearn.model_selection import RandomizedSearchCV                # to perform Cross-Validation
from sklearn.model_selection import GridSearchCV                      # ...


-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

**Step 2 : Import data** 

In [2]:
df = pd.read_csv('Dataframe with picks.csv')                          # to download the data from the file into a dataframe called df
df.head()                                                             # to view the top 5 rows 


,Unnamed: 0,SMILES_new,ExactMolWt,qed,FpDensityMorgan2,TPSA,NumHAcceptors,NumHDonors,MolLogP,FractionCSP3,...,Outcome,pIC50,InChIKey,desalted\demetalled,mixture,complex,changes,Clusters,fingerprint,ID
0,2,Cc1cc(C)n(-c2cc(NC(=O)CCN(C)C)nc(-c3ccc(C)o3)n...,368.196074,0.719563,1.962963,89.08,7.0,1.0,2.73776,0.368421,...,1,6.187087,AACWUFIIMOHGSO-UHFFFAOYSA-N,False,False,False,NaN,2,<rdkit.DataStructs.cDataStructs.ExplicitBitVec...,PICKS
1,4,Cc1ccc(NC(=O)C(COC(C)C)Oc2ncnc3c2cnn3-c2ncccc2...,467.147265,0.418807,2.000000,116.94,9.0,1.0,3.37842,0.272727,...,0,NaN,AAGISEXHOPCAHZ-UHFFFAOYSA-N,False,False,False,NaN,4,<rdkit.DataStructs.cDataStructs.ExplicitBitVec...,PICKS
2,5,Cc1ncoc1-c1nnc(SCCCN2CCC3(CC3c3ccccc3)C2)n1C,409.193631,0.431725,2.103448,59.98,7.0,0.0,4.14032,0.500000,...,1,5.420000,AAPXNHMQKBDDJN-UHFFFAOYNA-N,False,False,False,NaN,5,<rdkit.DataStructs.cDataStructs.ExplicitBitVec...,PICKS
3,6,CN(C)Cc1ccc(C(F)(F)F)cc1Oc1ccc(Cl)c(Cl)c1,363.040454,0.673225,1.695652,12.47,2.0,0.0,5.86610,0.250000,...,1,5.950782,AAQZZAVVFRIEOD-UHFFFAOYSA-N,False,False,False,NaN,6,<rdkit.DataStructs.cDataStructs.ExplicitBitVec...,PICKS
4,10,O=C(Cc1ccc(-n2cnnn2)cc1)N1CCN(CCc2ccc3c(c2)COC...,418.211724,0.605615,1.709677,76.38,7.0,0.0,1.62190,0.391304,...,1,5.602060,AAYRYIFNKRSGOF-UHFFFAOYSA-N,False,False,False,NaN,7,<rdkit.DataStructs.cDataStructs.ExplicitBitVec...,PICKS


**Step 3 : Identify anomalies or missing data**

In [14]:
df.dtypes # to tell the data type of each column

Unnamed: 0                    int64
SMILES_new                   object
ExactMolWt                  float64
qed                         float64
FpDensityMorgan2            float64
TPSA                        float64
NumHAcceptors               float64
NumHDonors                  float64
MolLogP                     float64
FractionCSP3                float64
NumRotatableBonds           float64
HeavyAtomCount              float64
NumAliphaticCarbocycles     float64
NumAromaticCarbocycles      float64
NumAliphaticHeterocycles    float64
NumAromaticHeterocycles     float64
NumAromaticRings            float64
Compound_name                object
Standard Relation            object
IC50_nM                     float64
Outcome                       int64
pIC50                       float64
InChIKey                     object
desalted\demetalled            bool
mixture                        bool
complex                        bool
changes                      object
Clusters                    

In [16]:
df['qed'].unique() # to test printing a unique value in the column 2 for qed

array([0.71956264, 0.41880677, 0.43172538, ..., 0.29413806, 0.56564395,
       0.48294608])

In [17]:
for col in df.columns:
    print(col, df[-df[col].isna()].shape)

Unnamed: 0 (1800, 30)
SMILES_new (1800, 30)
ExactMolWt (1800, 30)
qed (1800, 30)
FpDensityMorgan2 (1800, 30)
TPSA (1800, 30)
NumHAcceptors (1800, 30)
NumHDonors (1800, 30)
MolLogP (1800, 30)
FractionCSP3 (1800, 30)
NumRotatableBonds (1800, 30)
HeavyAtomCount (1800, 30)
NumAliphaticCarbocycles (1800, 30)
NumAromaticCarbocycles (1800, 30)
NumAliphaticHeterocycles (1800, 30)
NumAromaticHeterocycles (1800, 30)
NumAromaticRings (1800, 30)
Compound_name (1800, 30)
Standard Relation (1800, 30)
IC50_nM (1800, 30)
Outcome (1800, 30)
pIC50 (1341, 30)
InChIKey (1800, 30)
desalted\demetalled (1800, 30)
mixture (1800, 30)
complex (1800, 30)
changes (5, 30)
Clusters (1800, 30)
fingerprint (1800, 30)
ID (1800, 30)


-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

*Part 3 : Decison Tree*

In [11]:
# 1. dictionaries containing the results of the model, parameters and metrics per cluster

dict_metrics_dt = {'mcc' : [], 'acc' : []}

dict_y_pred_dt = {}

dict_y_eval_dt = {}

dict_alpha = {1: 0.0029045948283003553,
 2: 0.004414720430076305,
 3: 0.010889015898933208,
 4: 0.003028216376104856,
 5: 0.004237183638218164,
 6: 0.003520556546807517,
 7: 0.0027236001868144316,
 8: 0.005023855569293759,
 -1: 0.015799666854435235}



print('Results of the Decision Tree :')

# 2. to set the for loop of the one-cluster-out cross-validation

for i in set(df.Clusters):

    clf_dts = []           # an array to contain all the decision trees


    # 3. to split the dataset into evaluation and training
    
    evaluation = df[df.Clusters == i]         
    training = df[-(df.Clusters == i)]
    
    X_train = training[['ExactMolWt','qed','FpDensityMorgan2','TPSA','NumHAcceptors','NumHDonors','MolLogP','FractionCSP3','NumRotatableBonds','HeavyAtomCount', 'NumAliphaticCarbocycles', 'NumAromaticCarbocycles', 'NumAliphaticHeterocycles','NumAromaticHeterocycles','NumAromaticRings']]
    Y_train = training['Outcome']

    X_eval = evaluation[['ExactMolWt','qed','FpDensityMorgan2','TPSA','NumHAcceptors','NumHDonors','MolLogP','FractionCSP3','NumRotatableBonds','HeavyAtomCount', 'NumAliphaticCarbocycles', 'NumAromaticCarbocycles', 'NumAliphaticHeterocycles','NumAromaticHeterocycles','NumAromaticRings']]
    Y_eval = evaluation['Outcome']



    # 4. to build Decision Tree Classifier model

    clf_dt = DecisionTreeClassifier(random_state = 50)
    clf_dt = clf_dt.fit(X_train, Y_train)

    # 5. to perform a Cost Complexity Pruning

    path = clf_dt.cost_complexity_pruning_path(X_train, Y_train)               # to determine values for alpha 
    ccp_alphas, impurities = path.ccp_alphas, path.impurities                  # to extract different values for alpha
    ccp_alphas = ccp_alphas[:-1]                                               # to exclude the maximum value of alpha because it would prune all the tree leaving only the root

    # For each alpha we will append our model to a list : 
    
    
    #for ccp_alpha in ccp_alphas:                                                  
        #clf_dt = DecisionTreeClassifier(random_state=50, ccp_alpha=ccp_alpha)  # to assign every alpha to a specific decision tree 
        #clf_dt.fit(X_train, Y_train)                                           # to fit the decision trees to the training dataset 
        #clf_dts.append(clf_dt)                                                 # to add all the decision trees into the array
    


    # 6. accuracy vs alpha for training and testing sets

    train_acc = []         # an array containing the training balanced_accuracy_score of all the decision trees
    test_acc = []          # an array containing the testing balanced_accuracy_score of all the decision trees
 
    for c in clf_dts: 
        y_train_pred = c.predict(X_train)                                   # to predict the score of the training dataset 
        y_test_pred = c.predict(X_eval)                                     # to predict the score of the testing dataset
        train_acc.append(balanced_accuracy_score(y_train_pred, Y_train))    # to add training accuracy of all the decision trees into an array 
        test_acc.append(balanced_accuracy_score(y_test_pred, Y_eval))       # to add testing accuracy of all the decision trees into an array 

    #diff_acc = [abs(i-j) for i, j in zip(test_acc,train_acc)]
    #index_test_acc = diff_acc.index(min(diff_acc))
    
    #for alpha in ccp_alphas :                                               # to determine the alpha value corresponding to the maximum value of the testing dataset
        #if list(ccp_alphas).index(alpha) == index_test_acc :
            #dict_alpha[i] = alpha


    clf_dt_final = DecisionTreeClassifier(random_state=50, ccp_alpha=dict_alpha[i])
    clf_dt_final.fit(X_train, Y_train) 
    y_pred = clf_dt_final.predict(X_eval)                                            # to predict the response for the test dataset



    # 7. to add the results into the dictionaries 
    
    dict_y_pred_dt[i] = y_pred
    dict_y_eval_dt[i] = Y_eval

    mcc_dt= matthews_corrcoef(Y_eval, y_pred) 
    acc_dt= balanced_accuracy_score(Y_eval, y_pred, adjusted = False) 

    dict_metrics_dt['mcc'].append(mcc_dt)
    dict_metrics_dt['acc'].append(acc_dt)

    #plot_confusion_matrix(clf_dt, X_eval, Y_eval, display_labels=["Is a Blocker", "Is not a Blocker"]) 

    print(f'fold{i} - MCC: {mcc_dt}, ACC: {acc_dt}')
    print(f'fold{i} - Optimal Alpha : {dict_alpha[i]}')



# 8. Aggregate and Mean of the metrics 

agg_y_pred_dt = list(itertools.chain.from_iterable(dict_y_pred_dt.values()))
agg_y_eval_dt = list(itertools.chain.from_iterable(dict_y_eval_dt.values()))
    
agg_mcc_dt = matthews_corrcoef(agg_y_eval_dt, agg_y_pred_dt)
agg_acc_dt = balanced_accuracy_score(agg_y_eval_dt, agg_y_pred_dt, adjusted = False)

print(f' Aggregate MCC: {agg_mcc_dt} - Aggregate ACC {agg_acc_dt}')

mean_mcc_dt = statistics.mean(list(dict_metrics_dt['mcc']))
mean_acc_dt = statistics.mean(list(dict_metrics_dt['acc']))

print(f'MCC mean: {mean_mcc_dt} ; ACC mean : {mean_acc_dt}')



Results of the Decision Tree :
fold1 - MCC: 0.26839804517998356, ACC: 0.632992327365729
fold1 - Optimal Alpha : 0.0029045948283003553
fold2 - MCC: 0.20394534103468318, ACC: 0.6043810155990708
fold2 - Optimal Alpha : 0.004414720430076305
fold3 - MCC: 0.26439680538041194, ACC: 0.632198402690206
fold3 - Optimal Alpha : 0.010889015898933208
fold4 - MCC: 0.2807524417899684, ACC: 0.640153452685422
fold4 - Optimal Alpha : 0.003028216376104856
fold5 - MCC: 0.29641075567784275, ACC: 0.6466747646522928
fold5 - Optimal Alpha : 0.004237183638218164
fold6 - MCC: 0.2847207731332163, ACC: 0.636
fold6 - Optimal Alpha : 0.003520556546807517
fold7 - MCC: 0.22663591891647744, ACC: 0.6118931048551611
fold7 - Optimal Alpha : 0.0027236001868144316
fold8 - MCC: 0.07044373768981514, ACC: 0.5309983896940419
fold8 - Optimal Alpha : 0.005023855569293759
fold-1 - MCC: 0.11582175587333282, ACC: 0.558786078098472
fold-1 - Optimal Alpha : 0.015799666854435235
 Aggregate MCC: 0.23113930967761867 - Aggregate ACC 0.615

-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

*Part 4 : Random Forest*

In [12]:
# 1. dictionaries containing the results of the model, parameters and metrics per cluster

dict_metrics_rf = {'mcc' : [], 'acc' : []}

dict_max_depth = {1:25, 2: 25, 3: 25, 4: 25, 5: 25, 6: 25, 7: 50, 8: 50, -1: 25}
dict_max_features = {1: 4, 2: 4, 3: 4, 4: 4, 5: 4, 6: 4, 7: 4, 8: 4, -1: 4}
dict_min_samples_leaf = {1: 5, 2: 2, 3: 2, 4: 2, 5: 2, 6: 2, 7: 2, 8: 2, -1: 2}
dict_min_samples_split = {1: 12, 2: 8, 3: 10, 4: 8, 5: 8, 6: 10, 7: 10, 8: 8, -1: 8}
dict_n_estimators = {1: 500, 2: 800, 3: 1000, 4: 500, 5: 500, 6: 500, 7: 500, 8: 500, -1: 500}

dict_rf_cluster = {}

dict_y_pred_rf = {}

dict_y_eval_rf = {}

# 2. to set the for loop of the one-cluster-out cross-validation

for i in set(df.Clusters):

    # 3. to split the dataset into evaluation and training

    evaluation = df[df.Clusters == i]          
    training = df[-(df.Clusters == i)]

    X_train = training[['ExactMolWt','qed','FpDensityMorgan2','TPSA','NumHAcceptors','NumHDonors','MolLogP','FractionCSP3','NumRotatableBonds','HeavyAtomCount', 'NumAliphaticCarbocycles', 'NumAromaticCarbocycles', 'NumAliphaticHeterocycles','NumAromaticHeterocycles','NumAromaticRings']]
    Y_train = training['Outcome']

    X_eval = evaluation[['ExactMolWt','qed','FpDensityMorgan2','TPSA','NumHAcceptors','NumHDonors','MolLogP','FractionCSP3','NumRotatableBonds','HeavyAtomCount', 'NumAliphaticCarbocycles', 'NumAromaticCarbocycles', 'NumAliphaticHeterocycles','NumAromaticHeterocycles','NumAromaticRings']]
    Y_eval = evaluation['Outcome']



    # 4. to create the parameter grid to realize GridSearchCV
    
   # param_grid = {                                                                                                  
   # 'max_depth': [5, 25, 50, 75, 100],
   # 'max_features': [4],
   # 'min_samples_leaf': [2, 2, 5],
   # 'min_samples_split': [8, 10, 12],
   # 'n_estimators': [1, 500, 800, 1000]
   # }



    # 5. to build Random Forest Classifier models 

    rf = RandomForestClassifier(ccp_alpha=0.001419, random_state=50, max_depth=dict_max_depth[i], max_features=dict_max_features[i], min_samples_split=dict_min_samples_split[i], min_samples_leaf= dict_min_samples_leaf[i], n_estimators=dict_n_estimators[i])                                                 
    #grid_search = GridSearchCV(estimator = rf, param_grid = dict_best_param_cluster[i], cv = 3, n_jobs = -1, verbose = 2)            # to initiate the grid search model with 5-fold cross validation 
    #grid_search.fit(X_train, Y_train)                                                                                # to fit the grid search to the data
    
    rf.fit(X_train, Y_train)                                                                                         # to build a forest of trees from the training set(X,Y)
    y_pred = rf.predict(X_eval)                                                                                      # to predict the response for the test dataset                
    


    # 6. to add the results into the dictionaries

    dict_rf_cluster[i] = y_pred

    #dict_best_param_cluster[i] = grid_search.best_params_                                                                         

    mcc_rf= matthews_corrcoef(Y_eval, y_pred) 
    acc_rf= balanced_accuracy_score(Y_eval, y_pred, adjusted = False) 

    dict_y_pred_rf[i] = y_pred
    dict_y_eval_rf[i] = Y_eval   
    
    dict_metrics_rf['mcc'].append(mcc_rf)
    dict_metrics_rf['acc'].append(acc_rf)

    print(f'fold{i} - MCC: {mcc_rf}, ACC: {acc_rf}') 

# 7. Aggregate and Mean of the metrics 

agg_y_pred_rf = list(itertools.chain.from_iterable(dict_y_pred_rf.values()))
agg_y_eval_rf = list(itertools.chain.from_iterable(dict_y_eval_rf.values()))
    
agg_mcc_rf = matthews_corrcoef(agg_y_eval_rf, agg_y_pred_rf)
agg_acc_rf = balanced_accuracy_score(agg_y_eval_rf, agg_y_pred_rf, adjusted = False)

print(f' Aggregate MCC: {agg_mcc_rf} - Aggregate ACC {agg_acc_rf}')

mean_mcc_rf = statistics.mean(list(dict_metrics_rf['mcc']))
mean_acc_rf = statistics.mean(list(dict_metrics_rf['acc']))

print(f'MCC mean: {mean_mcc_rf} ; ACC mean : {mean_acc_rf}')   

fold1 - MCC: 0.30105739770096623, ACC: 0.6442455242966751
fold2 - MCC: 0.35500732783714833, ACC: 0.6726960947007412
fold3 - MCC: 0.35296972131460014, ACC: 0.6793820933165196
fold4 - MCC: 0.39492599031416875, ACC: 0.6992327365728901
fold5 - MCC: 0.3040133698755911, ACC: 0.6524445794108715
fold6 - MCC: 0.30983866769659335, ACC: 0.628
fold7 - MCC: 0.2696762959982108, ACC: 0.6353529171766625
fold8 - MCC: 0.19581549326569875, ACC: 0.5940016103059581
fold-1 - MCC: 0.17885327041772822, ACC: 0.5918930390492361
 Aggregate MCC: 0.32000567905130345 - Aggregate ACC 0.6600020493852462
MCC mean: 0.2957952816023006 ; ACC mean : 0.6441387327588394


-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------